In [1]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Modeling
import tensorflow as tf
from sklearn.linear_model import LinearRegression

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Processing
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import cross_val_score

from src.utils.time_series_procs import split_sequences
from src.models.transformer_ts_tf import Transformer

from sklearn.model_selection import train_test_split

In [2]:
def score_model(model, x, y, n_splits=10, n_repeats=10):
    scoring = "neg_root_mean_squared_error"
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=SEED)
    scores = cross_val_score(model, x, y, scoring=scoring, cv=cv, n_jobs=-1)
    return scores

In [3]:
def get_model(params):
    return Transformer(
        num_hid=params["num_features"],
        time_steps=params["timesteps"],
        num_head=params["num_heads"],
        num_layers_enc=params["num_layers_enc"],
        num_feed_forward=params["num_feed_forward"],
        units_output=params["units_output"]
    )

In [4]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [5]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow import keras

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1, activation="relu"):
        super().__init__()
        self.attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation=activation),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training):
        attn_output = self.attn(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class Transformer(keras.Model):
    def __init__(
            self,
            num_hid=64,  # embed_dim - num of features
            time_steps=7,
            num_head=2,
            num_feed_forward=128,  # pointwise dim
            num_layers_enc=4,
            dropout_rate=0.1,
            activation="relu",
            units_output = 1
    ):
        super().__init__()
        self.numlayers_enc = num_layers_enc
        self.enc_input = layers.Input((time_steps, num_hid))
        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward, dropout_rate, activation)
                for _ in range(num_layers_enc)
            ]
        )
        self.GlobalAveragePooling1D = layers.GlobalAveragePooling1D(data_format='channels_last')
        self.out = layers.Dense(units=units_output, activation='linear')

    def call(self, inputs):
        #x =  Time2Vector(x.shape[-1])
        x = self.encoder(inputs)
        x = self.GlobalAveragePooling1D(x)
        y = self.out(x)
        return y

In [6]:
def print_scores(scores, METRICS, METRICS_DICT):
    for phase in ["train", "test"]:
        print("******")
        print(f"[{phase.upper()}]")
        print("******")
        for metric in METRICS:
            name = METRICS_DICT[metric]           
            print(
                f"{name}: %.3f (%.3f)"
                % (
                    np.abs(np.mean(scores[f"{phase}_" + metric])),
                    np.std(scores[f"{phase}_" + metric]),
                )
            )
        print("\n======================\n")

In [7]:
def score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred):
    TRAIN_RMSE = mean_squared_error(y_true=y_train, y_pred=y_train_pred, squared=False)
    TRAIN_MAE = mean_absolute_error(y_true=y_train, y_pred=y_train_pred)
    TRAIN_MAPE = mean_absolute_percentage_error(y_true=y_train, y_pred=y_train_pred)
    TRAIN_R2 = r2_score(y_true=y_train, y_pred=y_train_pred)

    TEST_RMSE = mean_squared_error(y_true=y_test, y_pred=y_test_pred, squared=False)
    TEST_MAE = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
    TEST_MAPE = mean_absolute_percentage_error(y_true=y_test, y_pred=y_test_pred)
    TEST_R2 = r2_score(y_true=y_test, y_pred=y_test_pred)

    scores = {
        "train_neg_root_mean_squared_error": TRAIN_RMSE,
        "train_neg_mean_absolute_error": TRAIN_MAE,
        "train_neg_mean_absolute_percentage_error": TRAIN_MAPE,
        "train_r2": TRAIN_R2,
        "test_neg_root_mean_squared_error": TEST_RMSE,
        "test_neg_mean_absolute_error": TEST_MAE,
        "test_neg_mean_absolute_percentage_error": TEST_MAPE,
        "test_r2": TEST_R2,
    }
    return scores

In [8]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

In [9]:
df_clinker = pd.read_csv("../../../data/processed/partner_i-Oficial/clinker.csv")
df_cimento = pd.read_csv("../../../data/processed/partner_i-Oficial/cement-shipping.csv")

In [10]:
set(df_clinker.columns) - set(df_cimento.columns)

{'AR',
 'Aphthitalite',
 'C2S',
 'C3A cubic',
 'C3A ortho',
 'C3S M1',
 'C3S M3',
 'C3S Tot.',
 'C4AF',
 'Free lime.1',
 'LSF',
 'Liquid phase 1338',
 'Liquid phase 1450',
 'SR',
 'Sulphur alkali rate',
 'TiO2'}

In [11]:
set(df_cimento.columns) - set(df_clinker.columns)

{'%Clinker',
 '%Gypsum',
 '%Limestone',
 '1 day Compressive strength',
 '28 day Compressive strength',
 '3 day Compressive strength',
 '7 day Compressive strength',
 'Alite total',
 'Aluminate',
 'Aluminate cubic',
 'Aluminate orto',
 'Anhydrite',
 'Aphthalite',
 'Bassanite',
 'Belite alpha',
 'Belite beta',
 'Belite gamma',
 'Belite total',
 'Calcite',
 'Dehydration',
 'Dolimite',
 'Gypsum',
 'IR',
 'LOI',
 'Quartz',
 'Remarks_CPIIF40 Expedido',
 'Remarks_CPVARI Expedido',
 'Total alkali as Na2O'}

In [12]:
CLINKER_TO_DROP = [
    'AR',
     'Aphthitalite',
     'C2S',
     'C3A cubic',
     'C3A ortho',
     'C3S M1',
     'C3S M3',
     'C3S Tot.',
     'C4AF',
     'Free lime.1',
     'LSF',
     'Liquid phase 1338',
     'Liquid phase 1450',
     'SR',
     'Sulphur alkali rate',
     'TiO2'
]

CEMENT_TO_DROP = [
    '%Clinker',
    '%Gypsum',
    '%Limestone',
    '1 day Compressive strength',
    # '28 day Compressive strength',
    '3 day Compressive strength',
    '7 day Compressive strength',
    'Alite total',
    'Aluminate',
    'Aluminate cubic',
    'Aluminate orto',
    'Anhydrite',
    'Aphthalite',
    'Bassanite',
    'Belite alpha',
    'Belite beta',
    'Belite gamma',
    'Belite total',
    'Calcite',
    'Dehydration',
    'Dolimite',
    'Gypsum',
    'IR',
    'LOI',
    'Quartz',
    'Remarks_CPIIF40 Expedido',
    'Remarks_CPVARI Expedido',
    'Total alkali as Na2O'
]

In [13]:
df_clinker = df_clinker.drop(CLINKER_TO_DROP, axis=1)
df_cimento = df_cimento.drop(CEMENT_TO_DROP, axis=1)

In [14]:
df_clinker

,Date,CaO,MgO,Arcanite,Na2O,Al2O3,SiO2,SO3,K2O,Fe2O3,Free lime,Ferrite,Periclase,Portlandite
0,2020-01-11 15:00:00,64.51720,3.53330,0.61,0.2625,4.82520,19.86070,2.0278,1.3761,2.90930,6.18,6.730,2.500,0.170
1,2020-01-11 17:00:00,65.93120,3.67190,0.66,0.2414,4.73900,19.44350,1.5134,1.1837,2.98670,8.50,6.770,2.440,0.550
2,2020-01-11 19:00:00,64.76200,3.62370,0.97,0.2343,4.81290,20.27340,1.3333,1.1910,3.05860,3.47,7.200,2.620,0.240
3,2020-01-11 21:00:00,64.42930,3.54945,0.79,0.2402,4.77185,20.06305,1.6755,1.2726,3.06005,4.42,7.340,2.540,0.320
4,2020-01-11 23:00:00,64.09660,3.47520,0.91,0.2461,4.73080,19.85270,2.0177,1.3542,3.06150,5.37,7.110,2.640,0.540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150,2022-05-03 05:00:00,63.54070,4.30230,0.79,0.2725,4.63760,19.52540,1.6458,1.2908,3.26490,3.10,9.800,3.630,0.105
5151,2022-05-03 09:00:00,63.07030,4.20370,0.80,0.3073,4.65580,19.51010,1.8491,1.3412,3.21440,4.55,9.660,3.630,0.020
5152,2022-05-03 13:00:00,62.48160,4.23440,0.95,0.2872,4.72230,20.00800,1.5837,1.3319,3.31380,3.48,9.020,3.380,0.250
5153,2022-05-03 17:00:00,62.85425,4.28505,1.02,0.2757,4.75515,20.07075,1.6264,1.3342,3.32145,2.92,9.145,3.385,0.280


In [15]:
df_cimento

,Date,CaO,MgO,Na2O,Al2O3,SiO2,SO3,K2O,Fe2O3,Ferrite,Free lime,Portlandite,Periclase,Arcanite,28 day Compressive strength
0,2020-01-02,60.007980,3.67285,0.19218,4.27263,18.556379,4.43558,1.21612,2.79746,6.40,0.30,4.62,2.95,0.94,46.400002
1,2020-01-02,60.661800,2.53917,0.20305,4.30627,18.414921,2.54332,1.22146,3.03862,6.75,0.27,2.85,2.75,1.59,37.400002
2,2020-01-03,60.755032,2.57985,0.19014,4.15108,18.481070,2.98196,1.12279,3.07343,5.61,0.42,2.79,1.82,1.35,36.099998
3,2020-01-03,59.827351,3.71899,0.18423,4.25718,18.616140,4.42556,1.19938,2.76630,7.04,0.27,3.77,2.72,1.06,43.900002
4,2020-01-03,58.848289,2.56056,0.19986,4.18035,17.636169,4.07796,1.20883,3.03309,6.48,0.07,3.41,2.68,1.39,48.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,2022-04-27,59.845181,2.69335,0.19684,4.35530,18.473740,4.19836,1.21207,3.02851,8.78,0.87,0.48,3.15,0.69,47.000000
1230,2022-04-28,58.849140,2.25720,0.18503,4.08649,17.420740,3.76640,1.17111,2.93249,8.98,1.08,0.32,2.91,0.84,47.500000
1231,2022-04-29,60.535889,2.69857,0.19885,4.42368,18.693199,3.06924,1.21522,3.02881,8.49,0.39,0.58,2.91,0.82,37.000000
1232,2022-05-02,58.660820,2.58789,0.19424,4.16096,17.674860,4.07574,1.19196,3.04040,8.62,0.55,0.34,3.01,0.88,47.000000


In [16]:
COLUMNS_CLINKER = df_cimento.drop("28 day Compressive strength", axis=1).columns
COLUMNS_CEMENT  = df_cimento.columns

In [17]:
COLUMNS_CLINKER

Index(['Date', 'CaO', 'MgO', 'Na2O', 'Al2O3', 'SiO2', 'SO3', 'K2O', 'Fe2O3',
       'Ferrite', 'Free lime', 'Portlandite', 'Periclase', 'Arcanite'],
      dtype='object')

In [18]:
COLUMNS_CEMENT

Index(['Date', 'CaO', 'MgO', 'Na2O', 'Al2O3', 'SiO2', 'SO3', 'K2O', 'Fe2O3',
       'Ferrite', 'Free lime', 'Portlandite', 'Periclase', 'Arcanite',
       '28 day Compressive strength'],
      dtype='object')

# Linear Regression

In [19]:
y = df_cimento['28 day Compressive strength']
x = df_cimento.drop(['Date', '28 day Compressive strength'], axis=1)

In [20]:
model = LinearRegression()
scores = score_model(model, x, y, n_splits=10, n_repeats=10)

In [21]:
np.abs(np.mean(scores))

3.5345870856333392

In [22]:
x.shape

(1234, 13)

# Transformer

In [23]:
y = df_cimento['28 day Compressive strength']
x = df_cimento.drop(['Date', '28 day Compressive strength'], axis=1)
test_size = 0.2
x, y = split_sequences(pd.concat([x, y], axis=1).values, 3)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
sc = StandardScaler3DShape()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [24]:
params = {}
params["num_features"] = 13
params["timesteps"] = 3
params["num_heads"] = 2
params["num_layers_enc"] = 1
params["num_feed_forward"] = 32
params["units_output"] = 1

In [25]:
model = get_model(params)
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
)

model.fit(x_train, y_train, batch_size=32, epochs=200, verbose=1)

Epoch 1/200
31/31 [==============================] - 1s 4ms/step - loss: 1715.2654 - RMSE: 41.4158
Epoch 2/200
31/31 [==============================] - 0s 3ms/step - loss: 1577.7584 - RMSE: 39.7210
Epoch 3/200
31/31 [==============================] - 0s 3ms/step - loss: 1505.4209 - RMSE: 38.7998
Epoch 4/200
31/31 [==============================] - 0s 4ms/step - loss: 1436.4613 - RMSE: 37.9007
Epoch 5/200
31/31 [==============================] - 0s 4ms/step - loss: 1364.9071 - RMSE: 36.9446
Epoch 6/200
31/31 [==============================] - 0s 4ms/step - loss: 1289.4106 - RMSE: 35.9084
Epoch 7/200
31/31 [==============================] - 0s 3ms/step - loss: 1211.4340 - RMSE: 34.8057
Epoch 8/200
31/31 [==============================] - 0s 3ms/step - loss: 1129.6663 - RMSE: 33.6105
Epoch 9/200
31/31 [==============================] - 0s 3ms/step - loss: 1044.7505 - RMSE: 32.3226
Epoch 10/200
31/31 [==============================] - 0s 2ms/step - loss: 959.4602 - RMSE: 30.9752
Epoch 11/2

Epoch 85/200
31/31 [==============================] - 0s 3ms/step - loss: 24.2640 - RMSE: 4.9258
Epoch 86/200
31/31 [==============================] - 0s 4ms/step - loss: 24.0952 - RMSE: 4.9087
Epoch 87/200
31/31 [==============================] - 0s 4ms/step - loss: 24.2282 - RMSE: 4.9222
Epoch 88/200
31/31 [==============================] - 0s 3ms/step - loss: 24.2545 - RMSE: 4.9249
Epoch 89/200
31/31 [==============================] - 0s 2ms/step - loss: 23.9661 - RMSE: 4.8955
Epoch 90/200
31/31 [==============================] - 0s 2ms/step - loss: 23.8914 - RMSE: 4.8879
Epoch 91/200
31/31 [==============================] - 0s 2ms/step - loss: 24.0132 - RMSE: 4.9003
Epoch 92/200
31/31 [==============================] - 0s 2ms/step - loss: 24.7631 - RMSE: 4.9763
Epoch 93/200
31/31 [==============================] - 0s 3ms/step - loss: 24.5108 - RMSE: 4.9508
Epoch 94/200
31/31 [==============================] - 0s 4ms/step - loss: 24.1063 - RMSE: 4.9098
Epoch 95/200
31/31 [==========

31/31 [==============================] - 0s 3ms/step - loss: 23.2153 - RMSE: 4.8182
Epoch 169/200
31/31 [==============================] - 0s 3ms/step - loss: 23.0028 - RMSE: 4.7961
Epoch 170/200
31/31 [==============================] - 0s 3ms/step - loss: 22.6241 - RMSE: 4.7565
Epoch 171/200
31/31 [==============================] - 0s 4ms/step - loss: 22.4599 - RMSE: 4.7392
Epoch 172/200
31/31 [==============================] - 0s 3ms/step - loss: 22.8505 - RMSE: 4.7802
Epoch 173/200
31/31 [==============================] - 0s 3ms/step - loss: 23.0071 - RMSE: 4.7966
Epoch 174/200
31/31 [==============================] - 0s 2ms/step - loss: 22.8325 - RMSE: 4.7783
Epoch 175/200
31/31 [==============================] - 0s 3ms/step - loss: 23.0637 - RMSE: 4.8025
Epoch 176/200
31/31 [==============================] - 0s 2ms/step - loss: 22.8334 - RMSE: 4.7784
Epoch 177/200
31/31 [==============================] - 0s 2ms/step - loss: 22.9998 - RMSE: 4.7958
Epoch 178/200
31/31 [=============

In [26]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 4.735 (0.000)
MAE: 3.991 (0.000)
MAPE: 0.098 (0.000)
R2: 0.121 (0.000)


******
[TEST]
******
RMSE: 4.730 (0.000)
MAE: 4.157 (0.000)
MAPE: 0.098 (0.000)
R2: 0.094 (0.000)




In [27]:
model.summary()

Model: "transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 3, 13)             2372      
_________________________________________________________________
global_average_pooling1d (Gl multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  14        
Total params: 2,386
Trainable params: 2,386
Non-trainable params: 0
_________________________________________________________________


# Masked Clinker Model

In [28]:
import numpy as np
import keras
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, samples, batch_size=32, dim=(3,13), shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.samples = samples
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.samples) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        samples_temp = [self.samples[k].copy() for k in indexes]

        # Generate data
        X, y = self.__data_generation(samples_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.samples))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, samples_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=float)        
        samples_temp = [array.copy() for array in samples_temp]
        
        # Generate data
        for i, sample in enumerate(samples_temp):
            feature = np.random.randint(low=0, high=13, size=None, dtype=int)
            # Store sample
#             for j, _ in enumerate(sample):
#                 y[i] = sample[j][feature]
#                 sample[j][feature] = -100
            y[i] = sample[0][feature]
            sample[0][feature] = -100
            X[i,] = sample           

        return X, y

In [29]:
def data_generation(samples_temp):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    X = np.empty((32, *(3, 13)))
    y = np.empty((32, 13), dtype=float)        
    samples_temp_copy = [array.copy() for array in samples_temp]
    
    # Generate data
    for i, sample in enumerate(samples_temp_copy):
        feature = np.random.randint(low=0, high=X.shape[1], size=None, dtype=int)
        # Store sample
        y[i] = sample[feature, :]
        sample[feature, :] = -100
        X[i,] = sample
    return X, y

In [30]:
import numpy as np
from tensorflow import keras
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

class DataGenerator_2(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, samples, batch_size=32, dim=(3,13), shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.samples = samples
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.samples) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        samples_temp = [self.samples[k].copy() for k in indexes]

        # Generate data
        X, y = self.__data_generation(samples_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.samples))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, samples_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size, X.shape[-1]), dtype=float)        
        samples_temp = [array.copy() for array in samples_temp]
        
        # Generate data
        for i, sample in enumerate(samples_temp):
            #feature = np.random.randint(low=0, high=X.shape[1], size=None, dtype=int)
            feature = X.shape[1] - 1
            # Store sample            
            y[i] = sample[feature, :]
            #if np.random.choice([1, 0], p=[0.3, 0.7]) == 1:
            #    sample[feature, :] = -999
            sample[feature, :] = -999
            X[i,] = sample

        return X, y

## Pre Trainning

In [31]:
TIMESTEPS = 5
BATCH_SIZE = 64

In [32]:
x = df_clinker.drop("Date", axis=1)
y = df_clinker.iloc[:, 0]
x, _ = split_sequences(pd.concat([x, y], axis=1).values, TIMESTEPS)
sc = StandardScaler3DShape()
x = sc.fit_transform(x)

In [33]:
dtgen_params = {
    'dim': (TIMESTEPS, x.shape[-1]),
    'batch_size': BATCH_SIZE,
    'shuffle': True
}

training_generator = DataGenerator_2(x, **dtgen_params)

In [34]:
params = {}
params["num_features"] = x.shape[-1] # 13
params["timesteps"] = TIMESTEPS
params["num_heads"] = 5
params["num_layers_enc"] = 5
params["num_feed_forward"] = 128
params["units_output"] = x.shape[-1] # 13

model = get_model(params)
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
)

In [35]:
model.fit_generator(generator=training_generator, epochs=500, verbose=1)

/home/felipe/Documents/ccs28-venv/lib/python3.6/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
80/80 [==============================] - 8s 24ms/step - loss: 0.8238 - RMSE: 0.9077
Epoch 2/500
80/80 [==============================] - 2s 21ms/step - loss: 0.7305 - RMSE: 0.8547
Epoch 3/500
80/80 [==============================] - 3s 43ms/step - loss: 0.6966 - RMSE: 0.8346
Epoch 4/500
80/80 [==============================] - 2s 22ms/step - loss: 0.6780 - RMSE: 0.8234
Epoch 5/500
80/80 [==============================] - 2s 31ms/step - loss: 0.6621 - RMSE: 0.8137
Epoch 6/500
80/80 [==============================] - 2s 30ms/step - loss: 0.6570 - RMSE: 0.8106
Epoch 7/500
80/80 [==============================] - 3s 39ms/step - loss: 0.6427 - RMSE: 0.8017
Epoch 8/500
80/80 [==============================] - 3s 36ms/step - loss: 0.6427 - RMSE: 0.8017
Epoch 9/500
80/80 [==============================] - 2s 30ms/step - loss: 0.6387 - RMSE: 0.7992
Epoch 10/500
80/80 [==============================] - 3s 34ms/step - loss: 0.6327 - RMSE: 0.7954
Epoch 11/500
80/80 [===================

80/80 [==============================] - 2s 28ms/step - loss: 0.5652 - RMSE: 0.7518
Epoch 86/500
80/80 [==============================] - 2s 28ms/step - loss: 0.5604 - RMSE: 0.7486
Epoch 87/500
80/80 [==============================] - 2s 28ms/step - loss: 0.5603 - RMSE: 0.7486
Epoch 88/500
80/80 [==============================] - 2s 28ms/step - loss: 0.5595 - RMSE: 0.7480
Epoch 89/500
80/80 [==============================] - 3s 34ms/step - loss: 0.5593 - RMSE: 0.7479
Epoch 90/500
80/80 [==============================] - 2s 28ms/step - loss: 0.5579 - RMSE: 0.7469
Epoch 91/500
80/80 [==============================] - 2s 28ms/step - loss: 0.5558 - RMSE: 0.7455
Epoch 92/500
80/80 [==============================] - 2s 28ms/step - loss: 0.5542 - RMSE: 0.7444
Epoch 93/500
80/80 [==============================] - 2s 23ms/step - loss: 0.5548 - RMSE: 0.7448
Epoch 94/500
80/80 [==============================] - 2s 25ms/step - loss: 0.5525 - RMSE: 0.7433
Epoch 95/500
80/80 [=======================

80/80 [==============================] - 2s 26ms/step - loss: 0.5226 - RMSE: 0.7229
Epoch 169/500
80/80 [==============================] - 2s 26ms/step - loss: 0.5190 - RMSE: 0.7204
Epoch 170/500
80/80 [==============================] - 2s 27ms/step - loss: 0.5163 - RMSE: 0.7185
Epoch 171/500
80/80 [==============================] - 2s 26ms/step - loss: 0.5155 - RMSE: 0.7180
Epoch 172/500
80/80 [==============================] - 2s 25ms/step - loss: 0.5167 - RMSE: 0.7188
Epoch 173/500
80/80 [==============================] - 2s 26ms/step - loss: 0.5130 - RMSE: 0.7163
Epoch 174/500
80/80 [==============================] - 2s 26ms/step - loss: 0.5116 - RMSE: 0.7153
Epoch 175/500
80/80 [==============================] - 2s 25ms/step - loss: 0.5138 - RMSE: 0.7168
Epoch 176/500
80/80 [==============================] - 2s 25ms/step - loss: 0.5131 - RMSE: 0.7163
Epoch 177/500
80/80 [==============================] - 2s 26ms/step - loss: 0.5128 - RMSE: 0.7161
Epoch 178/500
80/80 [=============

80/80 [==============================] - 2s 20ms/step - loss: 0.4982 - RMSE: 0.7058
Epoch 252/500
80/80 [==============================] - 2s 20ms/step - loss: 0.4970 - RMSE: 0.7050
Epoch 253/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4938 - RMSE: 0.7027
Epoch 254/500
80/80 [==============================] - 2s 20ms/step - loss: 0.4957 - RMSE: 0.7040
Epoch 255/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4958 - RMSE: 0.7041
Epoch 256/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4917 - RMSE: 0.7012
Epoch 257/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4961 - RMSE: 0.7043
Epoch 258/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4930 - RMSE: 0.7021
Epoch 259/500
80/80 [==============================] - 2s 20ms/step - loss: 0.4993 - RMSE: 0.7066
Epoch 260/500
80/80 [==============================] - 2s 20ms/step - loss: 0.4968 - RMSE: 0.7048
Epoch 261/500
80/80 [=============

80/80 [==============================] - 2s 19ms/step - loss: 0.4830 - RMSE: 0.6950
Epoch 335/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4813 - RMSE: 0.6937
Epoch 336/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4796 - RMSE: 0.6925
Epoch 337/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4796 - RMSE: 0.6925
Epoch 338/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4784 - RMSE: 0.6916
Epoch 339/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4793 - RMSE: 0.6923
Epoch 340/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4808 - RMSE: 0.6934
Epoch 341/500
80/80 [==============================] - 2s 20ms/step - loss: 0.4792 - RMSE: 0.6922
Epoch 342/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4833 - RMSE: 0.6952
Epoch 343/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4821 - RMSE: 0.6943
Epoch 344/500
80/80 [=============

80/80 [==============================] - 2s 23ms/step - loss: 0.4623 - RMSE: 0.6799
Epoch 418/500
80/80 [==============================] - 2s 21ms/step - loss: 0.4582 - RMSE: 0.6769
Epoch 419/500
80/80 [==============================] - 2s 24ms/step - loss: 0.4612 - RMSE: 0.6791
Epoch 420/500
80/80 [==============================] - 2s 22ms/step - loss: 0.4581 - RMSE: 0.6768
Epoch 421/500
80/80 [==============================] - 2s 21ms/step - loss: 0.4622 - RMSE: 0.6798
Epoch 422/500
80/80 [==============================] - 2s 20ms/step - loss: 0.4599 - RMSE: 0.6781
Epoch 423/500
80/80 [==============================] - 2s 19ms/step - loss: 0.4662 - RMSE: 0.6828
Epoch 424/500
80/80 [==============================] - 2s 20ms/step - loss: 0.4679 - RMSE: 0.6840
Epoch 425/500
80/80 [==============================] - 1s 18ms/step - loss: 0.4634 - RMSE: 0.6808
Epoch 426/500
80/80 [==============================] - 2s 20ms/step - loss: 0.4625 - RMSE: 0.6801
Epoch 427/500
80/80 [=============

80/80 [==============================] - 1s 17ms/step - loss: 0.4593 - RMSE: 0.6777


# Fine Tunning

In [36]:
y = df_cimento['28 day Compressive strength']
x = df_cimento.drop(['Date', '28 day Compressive strength'], axis=1)
test_size = 0.2
x, y = split_sequences(pd.concat([x, y], axis=1).values, TIMESTEPS)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
sc = StandardScaler3DShape()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [37]:
for layer in model.layers:
    layer.trainable = False

In [38]:
model2 = tf.keras.Sequential(
    model.layers[:-1] +
    [
        tf.keras.layers.Dense(units=256, activation="relu"),
        tf.keras.layers.Dense(units=128, activation="relu"),
        layers.Dense(units=1, activation='linear')
    ]
)

In [39]:
#model = get_model(params)
model2.compile(
    tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
)

model2.fit(x_train, y_train, batch_size=32, epochs=200, verbose=1)

Epoch 1/200
31/31 [==============================] - 3s 5ms/step - loss: 1520.4382 - RMSE: 38.9928
Epoch 2/200
31/31 [==============================] - 0s 6ms/step - loss: 244.9743 - RMSE: 15.6517
Epoch 3/200
31/31 [==============================] - 0s 6ms/step - loss: 40.8714 - RMSE: 6.3931
Epoch 4/200
31/31 [==============================] - 0s 6ms/step - loss: 35.7350 - RMSE: 5.9779
Epoch 5/200
31/31 [==============================] - 0s 6ms/step - loss: 34.2473 - RMSE: 5.8521
Epoch 6/200
31/31 [==============================] - 0s 5ms/step - loss: 34.2381 - RMSE: 5.8513
Epoch 7/200
31/31 [==============================] - 0s 5ms/step - loss: 33.2623 - RMSE: 5.7673
Epoch 8/200
31/31 [==============================] - 0s 6ms/step - loss: 32.6273 - RMSE: 5.7120
Epoch 9/200
31/31 [==============================] - 0s 6ms/step - loss: 30.0668 - RMSE: 5.4833
Epoch 10/200
31/31 [==============================] - 0s 6ms/step - loss: 30.5445 - RMSE: 5.5267
Epoch 11/200
31/31 [==============

31/31 [==============================] - 0s 6ms/step - loss: 26.9680 - RMSE: 5.1931
Epoch 86/200
31/31 [==============================] - 0s 6ms/step - loss: 26.9601 - RMSE: 5.1923
Epoch 87/200
31/31 [==============================] - 0s 6ms/step - loss: 26.3744 - RMSE: 5.1356
Epoch 88/200
31/31 [==============================] - 0s 6ms/step - loss: 27.3143 - RMSE: 5.2263
Epoch 89/200
31/31 [==============================] - 0s 6ms/step - loss: 27.1695 - RMSE: 5.2124
Epoch 90/200
31/31 [==============================] - 0s 6ms/step - loss: 26.5642 - RMSE: 5.1540
Epoch 91/200
31/31 [==============================] - 0s 6ms/step - loss: 26.7146 - RMSE: 5.1686
Epoch 92/200
31/31 [==============================] - 0s 6ms/step - loss: 26.7321 - RMSE: 5.1703
Epoch 93/200
31/31 [==============================] - 0s 6ms/step - loss: 26.2218 - RMSE: 5.1207
Epoch 94/200
31/31 [==============================] - 0s 6ms/step - loss: 26.8506 - RMSE: 5.1818
Epoch 95/200
31/31 [=======================

31/31 [==============================] - 0s 5ms/step - loss: 28.6636 - RMSE: 5.3538
Epoch 169/200
31/31 [==============================] - 0s 6ms/step - loss: 27.4391 - RMSE: 5.2382
Epoch 170/200
31/31 [==============================] - 0s 5ms/step - loss: 26.2882 - RMSE: 5.1272
Epoch 171/200
31/31 [==============================] - 0s 5ms/step - loss: 26.3054 - RMSE: 5.1289
Epoch 172/200
31/31 [==============================] - 0s 6ms/step - loss: 25.9433 - RMSE: 5.0935
Epoch 173/200
31/31 [==============================] - 0s 6ms/step - loss: 27.2360 - RMSE: 5.2188
Epoch 174/200
31/31 [==============================] - 0s 6ms/step - loss: 26.1417 - RMSE: 5.1129
Epoch 175/200
31/31 [==============================] - 0s 6ms/step - loss: 27.2490 - RMSE: 5.2201
Epoch 176/200
31/31 [==============================] - 0s 6ms/step - loss: 26.5084 - RMSE: 5.1486
Epoch 177/200
31/31 [==============================] - 0s 6ms/step - loss: 25.7421 - RMSE: 5.0737
Epoch 178/200
31/31 [=============

In [40]:
y_train_pred = model2.predict(x_train)
y_test_pred = model2.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 5.025 (0.000)
MAE: 4.587 (0.000)
MAPE: 0.111 (0.000)
R2: 0.011 (0.000)


******
[TEST]
******
RMSE: 4.977 (0.000)
MAE: 4.735 (0.000)
MAPE: 0.112 (0.000)
R2: 0.010 (0.000)




In [41]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten

model.layers

In [42]:
encoder_layers = model.layers[0]
encoder_layers.trainable = True

visible = Input(shape=(TIMESTEPS, 13))
enc = encoder_layers(visible)
hidden1 = Dense(128, activation='relu')(enc)
hidden2 = Dense(64, activation='relu')(hidden1)
hidden3 = Dense(32, activation='relu')(hidden2)
hidden4 = Flatten()(hidden3)
output = Dense(1)(hidden4)
model_2 = Model(inputs=visible, outputs=output)

In [43]:
model_2.compile(
    tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
)

model_2.fit(x_train, y_train, batch_size=32, epochs=500, verbose=1)

Epoch 1/500
31/31 [==============================] - 4s 12ms/step - loss: 1307.6304 - RMSE: 36.1612
Epoch 2/500
31/31 [==============================] - 0s 12ms/step - loss: 70.6806 - RMSE: 8.4072
Epoch 3/500
31/31 [==============================] - 0s 12ms/step - loss: 26.3287 - RMSE: 5.1311
Epoch 4/500
31/31 [==============================] - 0s 12ms/step - loss: 26.1464 - RMSE: 5.1134
Epoch 5/500
31/31 [==============================] - 0s 12ms/step - loss: 25.4925 - RMSE: 5.0490
Epoch 6/500
31/31 [==============================] - 0s 12ms/step - loss: 26.8895 - RMSE: 5.1855
Epoch 7/500
31/31 [==============================] - 0s 13ms/step - loss: 25.8753 - RMSE: 5.0868
Epoch 8/500
31/31 [==============================] - 0s 12ms/step - loss: 25.6428 - RMSE: 5.0639
Epoch 9/500
31/31 [==============================] - 0s 13ms/step - loss: 25.7510 - RMSE: 5.0745
Epoch 10/500
31/31 [==============================] - 0s 12ms/step - loss: 26.0680 - RMSE: 5.1057
Epoch 11/500
31/31 [======

31/31 [==============================] - 0s 13ms/step - loss: 26.3816 - RMSE: 5.1363
Epoch 85/500
31/31 [==============================] - 0s 13ms/step - loss: 26.6457 - RMSE: 5.1620
Epoch 86/500
31/31 [==============================] - 0s 13ms/step - loss: 25.4144 - RMSE: 5.0413
Epoch 87/500
31/31 [==============================] - 0s 13ms/step - loss: 26.6412 - RMSE: 5.1615
Epoch 88/500
31/31 [==============================] - 0s 12ms/step - loss: 25.9683 - RMSE: 5.0959
Epoch 89/500
31/31 [==============================] - 0s 13ms/step - loss: 26.6014 - RMSE: 5.1576
Epoch 90/500
31/31 [==============================] - 0s 12ms/step - loss: 27.3709 - RMSE: 5.2317
Epoch 91/500
31/31 [==============================] - 0s 13ms/step - loss: 26.7822 - RMSE: 5.1752
Epoch 92/500
31/31 [==============================] - 0s 13ms/step - loss: 26.4456 - RMSE: 5.1425
Epoch 93/500
31/31 [==============================] - 0s 13ms/step - loss: 26.4213 - RMSE: 5.1402
Epoch 94/500
31/31 [=============

31/31 [==============================] - 0s 13ms/step - loss: 13.2043 - RMSE: 3.6338
Epoch 168/500
31/31 [==============================] - 0s 13ms/step - loss: 11.4733 - RMSE: 3.3872
Epoch 169/500
31/31 [==============================] - 0s 13ms/step - loss: 12.7881 - RMSE: 3.5760
Epoch 170/500
31/31 [==============================] - 0s 13ms/step - loss: 12.3039 - RMSE: 3.5077
Epoch 171/500
31/31 [==============================] - 0s 13ms/step - loss: 11.3299 - RMSE: 3.3660
Epoch 172/500
31/31 [==============================] - 0s 13ms/step - loss: 11.0145 - RMSE: 3.3188
Epoch 173/500
31/31 [==============================] - 0s 13ms/step - loss: 11.6992 - RMSE: 3.4204
Epoch 174/500
31/31 [==============================] - 0s 13ms/step - loss: 11.4890 - RMSE: 3.3895
Epoch 175/500
31/31 [==============================] - 0s 16ms/step - loss: 10.3285 - RMSE: 3.2138
Epoch 176/500
31/31 [==============================] - 0s 14ms/step - loss: 11.3672 - RMSE: 3.3715
Epoch 177/500
31/31 [===

31/31 [==============================] - 0s 13ms/step - loss: 4.2504 - RMSE: 2.0616
Epoch 251/500
31/31 [==============================] - 0s 13ms/step - loss: 5.8660 - RMSE: 2.4220
Epoch 252/500
31/31 [==============================] - 0s 13ms/step - loss: 5.6254 - RMSE: 2.3718
Epoch 253/500
31/31 [==============================] - 0s 13ms/step - loss: 5.1916 - RMSE: 2.2785
Epoch 254/500
31/31 [==============================] - 0s 13ms/step - loss: 5.2901 - RMSE: 2.3000
Epoch 255/500
31/31 [==============================] - 0s 13ms/step - loss: 4.4087 - RMSE: 2.0997
Epoch 256/500
31/31 [==============================] - 0s 14ms/step - loss: 4.0597 - RMSE: 2.0149
Epoch 257/500
31/31 [==============================] - 0s 13ms/step - loss: 4.5148 - RMSE: 2.1248
Epoch 258/500
31/31 [==============================] - 0s 13ms/step - loss: 4.4408 - RMSE: 2.1073
Epoch 259/500
31/31 [==============================] - 0s 15ms/step - loss: 4.6729 - RMSE: 2.1617
Epoch 260/500
31/31 [=============

31/31 [==============================] - 0s 13ms/step - loss: 3.4388 - RMSE: 1.8544
Epoch 334/500
31/31 [==============================] - 0s 14ms/step - loss: 3.2507 - RMSE: 1.8030
Epoch 335/500
31/31 [==============================] - 0s 15ms/step - loss: 4.2054 - RMSE: 2.0507
Epoch 336/500
31/31 [==============================] - 0s 13ms/step - loss: 4.5270 - RMSE: 2.1277
Epoch 337/500
31/31 [==============================] - 0s 13ms/step - loss: 4.3510 - RMSE: 2.0859
Epoch 338/500
31/31 [==============================] - 0s 13ms/step - loss: 3.5719 - RMSE: 1.8899
Epoch 339/500
31/31 [==============================] - 0s 13ms/step - loss: 3.0019 - RMSE: 1.7326
Epoch 340/500
31/31 [==============================] - 0s 13ms/step - loss: 3.2778 - RMSE: 1.8105
Epoch 341/500
31/31 [==============================] - 0s 13ms/step - loss: 3.5769 - RMSE: 1.8913
Epoch 342/500
31/31 [==============================] - 1s 17ms/step - loss: 3.8330 - RMSE: 1.9578
Epoch 343/500
31/31 [=============

31/31 [==============================] - 1s 20ms/step - loss: 1.8961 - RMSE: 1.3770
Epoch 417/500
31/31 [==============================] - 1s 18ms/step - loss: 2.0308 - RMSE: 1.4251
Epoch 418/500
31/31 [==============================] - 0s 16ms/step - loss: 2.7001 - RMSE: 1.6432
Epoch 419/500
31/31 [==============================] - 1s 17ms/step - loss: 3.0762 - RMSE: 1.7539
Epoch 420/500
31/31 [==============================] - 0s 15ms/step - loss: 2.8237 - RMSE: 1.6804
Epoch 421/500
31/31 [==============================] - 1s 17ms/step - loss: 2.6341 - RMSE: 1.6230
Epoch 422/500
31/31 [==============================] - 0s 15ms/step - loss: 2.7803 - RMSE: 1.6674
Epoch 423/500
31/31 [==============================] - 0s 15ms/step - loss: 2.9432 - RMSE: 1.7156
Epoch 424/500
31/31 [==============================] - 0s 15ms/step - loss: 2.4398 - RMSE: 1.5620
Epoch 425/500
31/31 [==============================] - 0s 14ms/step - loss: 2.3196 - RMSE: 1.5230
Epoch 426/500
31/31 [=============

31/31 [==============================] - 1s 40ms/step - loss: 2.0428 - RMSE: 1.4293
Epoch 500/500
31/31 [==============================] - 1s 36ms/step - loss: 1.7544 - RMSE: 1.3245


In [44]:
y_train_pred = model_2.predict(x_train)
y_test_pred = model_2.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 0.893 (0.000)
MAE: 0.683 (0.000)
MAPE: 0.016 (0.000)
R2: 0.969 (0.000)


******
[TEST]
******
RMSE: 3.821 (0.000)
MAE: 2.480 (0.000)
MAPE: 0.057 (0.000)
R2: 0.405 (0.000)


